https://python.langchain.com/docs/how_to/custom_tools/

In [1]:
from getpass import getpass

In [2]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

Enter your OpenAI Token: ········


In [3]:
from ase.units import kJ

In [4]:
from ase.build import bulk

In [5]:
from ase.calculators.emt import EMT
from ase.eos import calculate_eos

In [6]:
from ase.data import reference_states, atomic_numbers

In [7]:
from langchain.agents import tool

In [8]:
@tool
def get_crystal_structure(chemical_symbol: str) -> str:
    """Returns the atomic crystal structure of a chemcial symbol"""
    ref_state = reference_states[atomic_numbers[chemical_symbol]]
    if ref_state is None:
        return "No crystal structure known."
    else:
        return ref_state["symmetry"]

In [9]:
@tool
def get_bulk_modulus(chemical_symbol: str, crystal_structure: str) -> str:
    """Returns the bulk modulus of chemcial symbol for a given crystal structure in GPa"""
    atoms = bulk(name=chemical_symbol, crystalstructure=crystal_structure,)
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()
    return B / kJ * 1.0e24

In [10]:
el = "Al"
crystal_structure = get_crystal_structure.invoke("Al")
get_bulk_modulus.invoke({"chemical_symbol": "Al", "crystal_structure": crystal_structure})

39.91626202227382

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [12]:
tools = [get_crystal_structure, get_bulk_modulus]

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [14]:
llm_with_tools = llm.bind_tools(tools)

In [15]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
lst = list(agent_executor.stream({"input": "What is the bulk modulus of gold"}))  # Yeah this worked !!



> Entering new AgentExecutor chain...

Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`
responded: I can help you with that. Let me check the bulk modulus of gold for you.

fcc
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`


173.83898364629107The bulk modulus of gold in the face-centered cubic (fcc) crystal structure is approximately 173.84 GPa.

> Finished chain.


In [18]:
lst[0]

{'actions': [ToolAgentAction(tool='get_crystal_structure', tool_input={'chemical_symbol': 'Au'}, log="\nInvoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`\nresponded: I can help you with that. Let me check the bulk modulus of gold for you.\n\n", message_log=[AIMessageChunk(content='I can help you with that. Let me check the bulk modulus of gold for you.', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP', 'function': {'arguments': '{"chemical_symbol":"Au"}', 'name': 'get_crystal_structure'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-52bbe691-14aa-4c16-ab64-fdcb19e10aeb', tool_calls=[{'name': 'get_crystal_structure', 'args': {'chemical_symbol': 'Au'}, 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP'}], tool_call_chunks=[{'name': 'get_crystal_structure', 'args': '{"chemical_symbol":"Au"}', 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP', 'index': 0}])], tool_call_id='call_wEmppnlhVEDrNxR24oDQVnuP')],
 'messages': [AIMe

In [19]:
lst[1]

{'steps': [AgentStep(action=ToolAgentAction(tool='get_crystal_structure', tool_input={'chemical_symbol': 'Au'}, log="\nInvoking: `get_crystal_structure` with `{'chemical_symbol': 'Au'}`\nresponded: I can help you with that. Let me check the bulk modulus of gold for you.\n\n", message_log=[AIMessageChunk(content='I can help you with that. Let me check the bulk modulus of gold for you.', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP', 'function': {'arguments': '{"chemical_symbol":"Au"}', 'name': 'get_crystal_structure'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-52bbe691-14aa-4c16-ab64-fdcb19e10aeb', tool_calls=[{'name': 'get_crystal_structure', 'args': {'chemical_symbol': 'Au'}, 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP'}], tool_call_chunks=[{'name': 'get_crystal_structure', 'args': '{"chemical_symbol":"Au"}', 'id': 'call_wEmppnlhVEDrNxR24oDQVnuP', 'index': 0}])], tool_call_id='call_wEmppnlhVEDrNxR24oDQVnuP'), obse

In [20]:
lst[2]

{'actions': [ToolAgentAction(tool='get_bulk_modulus', tool_input={'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}, log="\nInvoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA', 'function': {'arguments': '{"chemical_symbol":"Au","crystal_structure":"fcc"}', 'name': 'get_bulk_modulus'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-05a2099e-37a8-4b2c-af50-843b6a1511d6', tool_calls=[{'name': 'get_bulk_modulus', 'args': {'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}, 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA'}], tool_call_chunks=[{'name': 'get_bulk_modulus', 'args': '{"chemical_symbol":"Au","crystal_structure":"fcc"}', 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA', 'index': 0}])], tool_call_id='call_f0oXjeOQGlMMlhKQYQQEj5sA')],
 'messages': [AIMessageChunk(content='', additional_kwargs={'t

In [21]:
lst[3]

{'steps': [AgentStep(action=ToolAgentAction(tool='get_bulk_modulus', tool_input={'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}, log="\nInvoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA', 'function': {'arguments': '{"chemical_symbol":"Au","crystal_structure":"fcc"}', 'name': 'get_bulk_modulus'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-05a2099e-37a8-4b2c-af50-843b6a1511d6', tool_calls=[{'name': 'get_bulk_modulus', 'args': {'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}, 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA'}], tool_call_chunks=[{'name': 'get_bulk_modulus', 'args': '{"chemical_symbol":"Au","crystal_structure":"fcc"}', 'id': 'call_f0oXjeOQGlMMlhKQYQQEj5sA', 'index': 0}])], tool_call_id='call_f0oXjeOQGlMMlhKQYQQEj5sA'), observation=173.83898364629107)],
 'messages': [

In [22]:
lst[4]

{'output': 'The bulk modulus of gold in the face-centered cubic (fcc) crystal structure is approximately 173.84 GPa.',
 'messages': [AIMessage(content='The bulk modulus of gold in the face-centered cubic (fcc) crystal structure is approximately 173.84 GPa.')]}

# Lessons learned: 
* It works surprisingly well